In [4]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

geo_data = pd.read_csv('distance_to_lake.csv')
feature_data = geo_data[['num','d_water','d_park',
       'd_lm','TL1','Tmin','Tmax']].dropna()
feature_data['sqrt_dw']=np.sqrt(feature_data['d_water'])
feature_data['sqrt_dp']=np.sqrt(feature_data['d_park'])
trainX=feature_data[['d_lm','TL1','Tmin','Tmax','sqrt_dw','sqrt_dp']]
trainy=feature_data[['num']].astype(float)

estimators = [10, 20, 50, 100, 200]
scores = []
#for n_est in estimators:
#    clf = RandomForestRegressor(n_estimators=n_est, random_state=42, cv=5,max_depth = 5)
#    score = np.mean(cross_val_score(clf, trainX, trainy))
#    scores.append(score)
#    print(n_est)

In [14]:
latmin = min(geo_data.lat)
latmax = max(geo_data.lat)
lonmin = min(geo_data.lon)
lonmax = max(geo_data.lon)

grd = np.indices((51,51)).T.reshape(-1,2)
grd = grd.astype(float)
grd[:,1] = (1/50.)*grd[:,1]*(lonmax - lonmin) + lonmin
grd[:,0] = (1/50.)*grd[:,0]*(latmax - latmin) + latmin
xx = np.linspace(lonmin,lonmax,51)
yy = np.linspace(latmin,latmax,51)

geogrid = pd.read_csv('gridlake.csv')
testX = pd.DataFrame(geogrid['d_lm'])
testX['TL1'] = 75.18
testX['sqrt_dw']=np.sqrt(geogrid['d_water'])
testX['sqrt_dp']=np.sqrt(geogrid['d_park'])
testX['Tmax'] = 86.63 
testX['Tmin'] = 65.18
testX = testX[['d_lm','TL1','Tmin','Tmax','sqrt_dw','sqrt_dp']]
rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(trainX,trainy)

YY = rfr.predict(testX)
testY = YY.reshape(51,51).transpose()

/Users/ianbakst/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [15]:
import branca
import folium
from folium import plugins
from shapely.geometry import Point, Polygon
import geojsoncontour
import matplotlib.pyplot as plt


colors = ['darkgreen','green','limegreen','yellow','gold','orange','red','darkred']
vmin = 0
vmax = 50
levels = len(colors)
cm = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)

cfig = plt.contourf(xx, yy, testY, levels, alpha=1.0, colors=colors, linestyles='None', vmin=vmin, vmax=vmax)
plt.savefig('map.png',dpi=300)
geojson = geojsoncontour.contourf_to_geojson(contourf=cfig,
                                             min_angle_deg=0.0,
                                             ndigits=5,stroke_width=0.5,
                                             fill_opacity=1.0)

geomap = folium.Map(location=[41.89, -87.64], zoom_start=13, tiles="stamenterrain")
folium.GeoJson(geojson,style_function=lambda x:{'color': x['properties']['stroke'],
                   'weight': x['properties']['stroke-width'],
                   'fillColor': x['properties']['fill'],
                   'opacity':  1.0}).add_to(geomap)
cm.caption = 'Mosquito Activity'
geomap.add_child(cm)
plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)
geomap.save(f'rf_activity_map.html')
plt.close()

In [11]:
trainX

,d_lm,TL1,Tmin,Tmax,sqrt_dw,sqrt_dp
0,0.030931,68.0,56.0,90.0,0.129137,0.027096
1,0.162091,68.0,56.0,90.0,0.136348,0.063283
2,0.030931,68.0,56.0,90.0,0.129137,0.027096
3,0.071573,68.0,56.0,90.0,0.011400,0.000000
4,0.098231,68.0,56.0,90.0,0.080034,0.014381
...,...,...,...,...,...,...
24765,0.162091,65.0,61.0,93.0,0.136348,0.063283
24766,0.060025,65.0,61.0,93.0,0.098772,0.031207
24767,0.034664,65.0,61.0,93.0,0.022931,0.056370
24768,0.128273,65.0,61.0,93.0,0.156301,0.053425
